# Config

In [1]:
# Set dark mode as default
import matplotlib.pyplot as plt
import plotly.io as pio
plt.style.use("dark_background")
pio.templates.default = "plotly_dark"

# Imports
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA

# Global variables
REPO_DIR: Path = Path("..").resolve()
assert REPO_DIR.name == "market-regime-classification", "`REPO_DIR` points to incorrect folder."
DATA_PATH: Path = REPO_DIR / "data/preprocessed"

# Data

### data import

In [3]:
# Data import (with labels)
df = (
    pd.read_csv(DATA_PATH / "daily_prices_with_labels.csv")
    .set_index("trading_date")
    .sort_index()
)

### data transformation

In [5]:
df

,delivery_date,open,close,target
trading_date,,,,
2015-11-30,2016-01-01,17.85,18.150,-1.0
2015-12-01,2016-01-01,18.05,18.100,-1.0
2015-12-02,2016-01-01,18.15,17.675,-1.0
2015-12-03,2016-01-01,17.64,17.375,-1.0
2015-12-04,2016-01-01,17.35,17.450,-1.0
...,...,...,...,...
2024-11-22,2024-12-01,48.80,46.935,-1.0
2024-11-25,2024-12-01,47.05,47.910,0.0
2024-11-26,2024-12-01,47.50,46.575,0.0
